In [7]:
pip install pandas requests pubchempy tqdm rdkit

  Using cached PubChemPy-1.0.4.tar.gz (29 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=96be2eaf55ad92d27d63bc9d0cbe414cb66e6861aa673051814fd1b390723eea
  Stored in directory: /Users/sungjun/Library/Caches/pip/wheels/78/0f/d0/080f82ce0d7fdc771401b6acac304bd2ee77d67dee34737bd6
Successfully built pubchempy
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import requests
import urllib.parse
import numpy as np
import random 
from pubchempy import get_compounds
import time
from tqdm import tqdm
import concurrent.futures
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, Crippen
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from rdkit.Chem import AllChem, Descriptors3D
from rdkit.Chem import DataStructs


# 0. train data 불러오기

In [12]:
ch_df=pd.read_csv('train.csv')

In [13]:
ch_df=ch_df[['Smiles','IC50_nM','pIC50']]

In [14]:
ch_df

,Smiles,IC50_nM,pIC50
0,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...,0.022,10.66
1,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,0.026,10.59
2,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,0.078,10.11
3,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,0.081,10.09
4,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...,0.099,10.00
...,...,...,...
1947,O=C(Nc1nc2cc[nH]cc-2n1)c1cccc([N+](=O)[O-])c1,30000.000,4.52
1948,CCCCn1c(NC(=O)c2cccc(Cl)c2)nc2ccccc21,30000.000,4.52
1949,O=C(Nc1nc2cc(F)c(F)cc2[nH]1)c1cccc([N+](=O)[O-...,30000.000,4.52
1950,OC[C@H]1C[C@@H](Nc2nc(Nc3ccccc3)ncc2-c2nc3cccc...,42000.000,4.38


# 1. PubChem에서 feature 가져오기

In [15]:
# SMILES 열에 있는 화합물의 정보를 가져오는 첫 번째 방법 (PubChemPy)
def get_compound_data_pubchempy(smiles, retries=3):
    compound_info = {}
    for _ in range(retries):
        try:
            compounds = get_compounds(smiles, namespace='smiles')
            if compounds:
                compound = compounds[0]
                compound_info = {
                    'CID': compound.cid,
                    'IUPAC Name': compound.iupac_name,
                    'Molecular Formula': compound.molecular_formula,
                    'Molecular Weight': compound.molecular_weight,
                }

                optional_attributes = [
                    ('Exact Mass', 'exact_mass'),
                    ('XLogP', 'xlogp'),
                    ('Hydrogen Bond Donors', 'h_bond_donor_count'),
                    ('Hydrogen Bond Acceptors', 'h_bond_acceptor_count'),
                    ('Rotatable Bonds', 'rotatable_bond_count'),
                    ('TPSA', 'tpsa'),
                    ('Complexity', 'complexity'),
                    ('Charge', 'charge'),
                    ('Heavy Atom Count', 'heavy_atom_count'),
                    ('Isotope Atom Count', 'isotope_atom_count'),
                    ('Defined Atom Stereocenter Count', 'defined_atom_stereocenter_count'),
                    ('Undefined Atom Stereocenter Count', 'undefined_atom_stereocenter_count'),
                    ('Defined Bond Stereocenter Count', 'defined_bond_stereocenter_count'),
                    ('Undefined Bond Stereocenter Count', 'undefined_bond_stereocenter_count'),
                    ('Covalently-Bonded Unit Count', 'covalent_unit_count'),
                ]

                for label, attr in optional_attributes:
                    value = getattr(compound, attr, None)
                    if value is not None:
                        compound_info[label] = value

                break  # 성공하면 루프 종료
        except Exception as e:
            print(f"Error with PubChemPy: {e} | SMILES: {smiles}")
            time.sleep(2)
    
    return compound_info

# SMILES 열에 있는 화합물의 정보를 가져오는 두 번째 방법 (PubChem REST API)
def get_compound_data_api(smiles, retries=3):
    compound_info = {}
    encoded_smiles = urllib.parse.quote(smiles)
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{encoded_smiles}/record/JSON"
    
    for _ in range(retries):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                compound_data = data['PC_Compounds'][0]

                # CID
                compound_info['CID'] = compound_data['id']['id']['cid']

                # 분자식, 분자량 등 주요 속성
                for prop in compound_data.get('props', []):
                    label = prop['urn']['label']
                    name = prop['urn'].get('name', '')
                    value = prop['value']

                    if 'sval' in value:
                        compound_info[f"{label} {name}"] = value['sval']
                    elif 'fval' in value:
                        compound_info[f"{label} {name}"] = value['fval']
                    elif 'ival' in value:
                        compound_info[f"{label} {name}"] = value['ival']
                
                break  # 성공하면 루프 종료
        except requests.exceptions.RequestException as e:
            print(f"Error with API request: {e} | SMILES: {smiles}")
            time.sleep(2)
    
    return compound_info

# 데이터를 병합하여 최종 데이터프레임 생성
def fetch_and_merge_smiles_data(ch_df, max_workers=6):
    all_compound_data = []

    def process_smiles(smiles):
        pubchempy_data = get_compound_data_pubchempy(smiles)
        api_data = get_compound_data_api(smiles)
        merged_data = {**pubchempy_data, **api_data}
        merged_data['Smiles'] = smiles
        return merged_data

    smiles_list = ch_df['Smiles'].tolist()

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        for result in tqdm(executor.map(process_smiles, smiles_list), total=len(smiles_list)):
            all_compound_data.append(result)

    merged_df = pd.DataFrame(all_compound_data)
    final_df = pd.merge(ch_df, merged_df, on='Smiles', how='left')

    return final_df

# ch_df 데이터프레임을 불러온 후 사용
# ch_df = pd.read_csv('path_to_your_csv.csv')
# 아래 코드를 사용하여 데이터 수집 및 병합
final_df = fetch_and_merge_smiles_data(ch_df)

 13%|█▎        | 248/1952 [01:33<06:56,  4.09it/s]

Error with PubChemPy: 'PUGREST.ServerBusy' | SMILES: CC(C)(O)[C@H](F)CNC(=O)c1cnc(-n2ncc3cc(C#N)cnc32)cc1NC1CCC(F)(F)C1


 24%|██▎       | 459/1952 [02:54<09:57,  2.50it/s]

Error with PubChemPy: 'PUGREST.ServerBusy' | SMILES: CN1CCC(n2cc(Nc3nc(NC4(C)CC4)c4c(=O)n(-c5cncnc5)ccc4n3)cn2)CC1


 24%|██▍       | 472/1952 [02:58<06:25,  3.83it/s]

...

 65%|██████▌   | 1278/1952 [08:06<02:57,  3.79it/s]

Error with PubChemPy: 'PUGREST.ServerBusy' | SMILES: COC(=O)N1CCN([C@H]2CC[C@H](Nc3nc(Nc4cnn(C)c4)nc4ccc(C(C)(C)C#N)nc34)CC2)CC1


 69%|██████▉   | 1344/1952 [08:32<03:19,  3.04it/s]

Error with PubChemPy: 'PUGREST.ServerBusy' | SMILES: NC(=O)[C@@H]1[C@H](Nc2nc(Nc3ccc(OCCN4CCCC4)cc3)ncc2-c2cccnc2)[C@H]2C=C[C@@H]1C2


100%|██████████| 1952/1952 [12:24<00:00,  2.62it/s]


# 2. Rdkit계산으로 추출한 변수

In [18]:

# 특성을 저장할 빈 리스트 생성
features = []

# 각 SMILES 문자열에 대해 RDKit으로 특성을 계산
for smiles in ch_df['Smiles']:
    molecule = Chem.MolFromSmiles(smiles)
    if molecule:
        # MorganGenerator를 사용하여 복잡성 계산
        morgan_gen = GetMorganGenerator(radius=2)
        morgan_fp = morgan_gen.GetFingerprint(molecule)
        
        # 특성 계산
        info = {
            "Exact Mass": Descriptors.ExactMolWt(molecule),
            "XLogP": Crippen.MolLogP(molecule),
            "Hydrogen Bond Donors": Lipinski.NumHDonors(molecule),
            "Hydrogen Bond Acceptors": Lipinski.NumHAcceptors(molecule),
            "Rotatable Bonds": Descriptors.NumRotatableBonds(molecule),
            "TPSA": rdMolDescriptors.CalcTPSA(molecule),
            "Charge": Chem.GetFormalCharge(molecule),
            "Heavy Atom Count": Descriptors.HeavyAtomCount(molecule),
            "Covalently-Bonded Unit Count": len(Chem.GetMolFrags(molecule)),
        }
    else:
        # 만약 SMILES가 유효하지 않은 경우, NaN 값으로 채움
        info = {key: float('nan') for key in [
            "Exact Mass", "XLogP", "Hydrogen Bond Donors", 
            "Hydrogen Bond Acceptors", "Rotatable Bonds", 
            "TPSA", "Charge", "Heavy Atom Count", 
            "Covalently-Bonded Unit Count"
        ]}
    
    features.append(info)

# 특성을 DataFrame으로 변환하고 원본 DataFrame과 병합
features_df = pd.DataFrame(features)
result_df = pd.concat([ch_df, features_df], axis=1)

In [19]:
#RDkit로 추가할 수 있는 feature들 추가.


# 함수: 분자식으로부터 다양한 지표를 계산하는 함수
def calculate_descriptors(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return None
    
    # Morgan Fingerprint 생성기 설정
    morgan_gen = GetMorganGenerator(radius=2, fpSize=2048)
    
    # 지표 계산
    descriptors = {
        'Smiles': smiles,
        'Complexity by RDkit': Descriptors.BertzCT(molecule),
        'Covalently-Bonded Unit Count by RDkit': len(Chem.GetMolFrags(molecule)),
        'Compound Complexity (Morgan) by RDkit': morgan_gen.GetFingerprint(molecule).GetNumOnBits(),
        'Fraction of sp3 Carbons by RDkit': Descriptors.FractionCSP3(molecule),
        'Ring Count by RDkit': Descriptors.RingCount(molecule),
        'Aromatic Ring Count by RDkit': Descriptors.NumAromaticRings(molecule),
        'Heteroatom Count by RDkit': Descriptors.NumHeteroatoms(molecule),
        'BalabanJ Index by RDkit': Descriptors.BalabanJ(molecule),
    }
    
    return descriptors

# 데이터프레임에 지표 계산 적용
def create_descriptors_dataframe(ch_df):
    calculated_data = []
    for smiles in ch_df['Smiles']:
        descriptors = calculate_descriptors(smiles)
        if descriptors:
            calculated_data.append(descriptors)
    
    return pd.DataFrame(calculated_data)

# ch_df 데이터프레임을 불러온 후 사용
rdkit_df = create_descriptors_dataframe(ch_df)
rdkit_df

,Smiles,Complexity by RDkit,Covalently-Bonded Unit Count by RDkit,Compound Complexity (Morgan) by RDkit,Fraction of sp3 Carbons by RDkit,Ring Count by RDkit,Aromatic Ring Count by RDkit,Heteroatom Count by RDkit,BalabanJ Index by RDkit
0,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...,2470.627440,1,114,0.566038,7,3,19,0.858127
1,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,1405.892793,1,76,0.500000,6,3,11,1.295765
2,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,1408.871656,1,78,0.481481,6,3,12,1.295765
3,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,1382.863661,1,73,0.461538,5,3,13,1.446176
4,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...,2454.627995,1,111,0.509804,6,4,16,0.894084
...,...,...,...,...,...,...,...,...,...
1947,O=C(Nc1nc2cc[nH]cc-2n1)c1cccc([N+](=O)[O-])c1,767.959163,1,43,0.000000,3,1,8,1.847029
1948,CCCCn1c(NC(=O)c2cccc(Cl)c2)nc2ccccc21,841.153910,1,47,0.222222,3,3,5,1.935112
1949,O=C(Nc1nc2cc(F)c(F)cc2[nH]1)c1cccc([N+](=O)[O-...,899.917121,1,45,0.000000,3,3,9,1.877989
1950,OC[C@H]1C[C@@H](Nc2nc(Nc3ccccc3)ncc2-c2nc3cccc...,1185.723794,1,56,0.260870,5,4,9,1.516899


In [20]:


# 소수점 두 번째 자리에서 6 이하면 버리고, 7 이상이면 올리는 함수 정의
def round_custom(value, precision):
    if np.isnan(value):
        return np.nan
    if precision == 1:
        # 소수점 두 번째 자리를 기준으로 반올림 (기존 로직)
        decimal_value = int(value * 100) % 10  # 소수점 두 번째 자리 확인
        if decimal_value >= 7:
            return np.ceil(value * 10) / 10  # 소수점 첫 자리로 올림
        else:
            return np.floor(value * 10) / 10  # 소수점 첫 자리로 버림
    elif precision == 2:
        # 소수점 세 번째 자리를 기준으로 반올림 (새로운 로직)
        decimal_value = int(value * 1000) % 10  # 소수점 세 번째 자리 확인
        if decimal_value >= 7:
            return np.ceil(value * 100) / 100  # 소수점 두 자리로 올림
        else:
            return np.floor(value * 100) / 100  # 소수점 두 자리로 버림

# 바꿀 행들 리스트 (소수점 두 번째 자리 처리)
rows_to_modify_two_decimal = [19, 26, 28, 43, 57, 204, 298, 336, 354, 745, 988, 1015, 1028, 
                  1062, 1114, 1142, 1213, 1647, 1713]

# 바꿀 행들 리스트 (소수점 세 번째 자리 처리 -> 두 자리로 반올림)
rows_to_modify_three_decimal = [995, 1057, 1161, 1398]

# 소수점 두 번째 자리에서 6 이하면 버리고, 7 이상이면 올림
for idx in rows_to_modify_two_decimal:
    result_value = result_df.loc[idx, 'XLogP']
    
    # 소수점 두 번째 자리 기준 처리
    rounded_value = round_custom(result_value, precision=1)
    
    # 대체한 값을 final_df에 넣어줌
    final_df.loc[idx, 'XLogP'] = rounded_value

# 소수점 세 번째 자리에서 두 자리로 반올림
for idx in rows_to_modify_three_decimal:
    result_value = result_df.loc[idx, 'XLogP']
    
    # 소수점 세 번째 자리 기준 처리
    rounded_value = round_custom(result_value, precision=2)
    
    # 대체한 값을 final_df에 넣어줌
    final_df.loc[idx, 'XLogP'] = rounded_value

In [21]:
# ch_df에서 추출할 열
ch_columns = ['Smiles', 'IC50_nM', 'pIC50']
ch_selected = ch_df[ch_columns]

# final_df에서 추출할 열
final_columns = ['Exact Mass', 'XLogP', 'Hydrogen Bond Donors', 'Hydrogen Bond Acceptors',
                 'Rotatable Bonds', 'TPSA', 'Charge', 'Heavy Atom Count', 'Covalently-Bonded Unit Count']
final_selected = final_df[final_columns]

# rdkit_df에서 추출할 열
rdkit_columns = ['Complexity by RDkit','Covalently-Bonded Unit Count by RDkit', 
                 'Compound Complexity (Morgan) by RDkit', 'Fraction of sp3 Carbons by RDkit', 
                 'Ring Count by RDkit', 'Aromatic Ring Count by RDkit', 'Heteroatom Count by RDkit', 'BalabanJ Index by RDkit']
rdkit_selected = rdkit_df[rdkit_columns]

# 데이터프레임을 지정된 순서대로 합치기 (열 기준으로)
df = pd.concat([ch_selected, final_selected, rdkit_selected], axis=1)

In [22]:
df

,Smiles,IC50_nM,pIC50,Exact Mass,XLogP,Hydrogen Bond Donors,Hydrogen Bond Acceptors,Rotatable Bonds,TPSA,Charge,Heavy Atom Count,Covalently-Bonded Unit Count,Complexity by RDkit,Covalently-Bonded Unit Count by RDkit,Compound Complexity (Morgan) by RDkit,Fraction of sp3 Carbons by RDkit,Ring Count by RDkit,Aromatic Ring Count by RDkit,Heteroatom Count by RDkit,BalabanJ Index by RDkit
0,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...,0.022,10.66,994.51640508,2.8,6,13,24,251.0,0,72,1,2470.627440,1,114,0.566038,7,3,19,0.858127
1,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,0.026,10.59,535.27071613,1.4,2,8,7,106.0,0,39,1,1405.892793,1,76,0.500000,6,3,11,1.295765
2,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,0.078,10.11,537.24998069,0.2,2,9,7,116.0,0,39,1,1408.871656,1,78,0.481481,6,3,12,1.295765
3,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...,0.081,10.09,545.23622233,1.3,2,10,8,106.0,0,39,1,1382.863661,1,73,0.461538,5,3,13,1.446176
4,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...,0.099,10.00,935.46153323,7.5,5,11,23,243.0,0,67,1,2454.627995,1,111,0.509804,6,4,16,0.894084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,O=C(Nc1nc2cc[nH]cc-2n1)c1cccc([N+](=O)[O-])c1,30000.000,4.52,283.07053917,1.4,2,5,2,117.0,0,21,1,767.959163,1,43,0.000000,3,1,8,1.847029
1948,CCCCn1c(NC(=O)c2cccc(Cl)c2)nc2ccccc21,30000.000,4.52,327.1138399,4.4,1,2,5,46.9,0,23,1,841.153910,1,47,0.222222,3,3,5,1.935112
1949,O=C(Nc1nc2cc(F)c(F)cc2[nH]1)c1cccc([N+](=O)[O-...,30000.000,4.52,318.05644645,2.7,2,6,2,104.0,0,23,1,899.917121,1,45,0.000000,3,3,9,1.877989
1950,OC[C@H]1C[C@@H](Nc2nc(Nc3ccccc3)ncc2-c2nc3cccc...,42000.000,4.38,449.15216079,3.6,5,9,6,152.0,0,32,1,1185.723794,1,56,0.260870,5,4,9,1.516899


# 3. 여러가지 3D 구조열 변수 추출

In [23]:
# SMILES 문자열에 수소 원자를 추가하는 함수
def add_hydrogens(smiles):
    mol = Chem.MolFromSmiles(smiles)  # SMILES 문자열을 Mol 객체로 변환
    mol_with_hs = Chem.AddHs(mol)     # 수소 원자 추가
    return Chem.MolToSmiles(mol_with_hs)  # Mol 객체를 SMILES 문자열로 변환

# X_combined 데이터프레임에 수소 원자를 추가한 SMILES 열 생성
df['Smiles'] = df['Smiles'].apply(add_hydrogens)

In [24]:

# 랜덤 시드 고정
seed = 42
random.seed(seed)
np.random.seed(seed)

# 3D 서술자를 계산하는 함수 정의
def compute_3d_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # 수소 추가
    
    # 시드를 고정하여 3D 구조 생성
    AllChem.EmbedMolecule(mol, randomSeed=seed)  # 3D 구조 생성 시 시드 적용
    AllChem.UFFOptimizeMolecule(mol)  # 구조 최적화

    # 3D 서술자 계산
    descriptors = {
        'Asphericity': Descriptors3D.Asphericity(mol),
        'Eccentricity': Descriptors3D.Eccentricity(mol),
        'SpherocityIndex': Descriptors3D.SpherocityIndex(mol),
        'PMI1': Descriptors3D.PMI1(mol),
        'PMI2': Descriptors3D.PMI2(mol),
        'PMI3': Descriptors3D.PMI3(mol),
        'NPR1': Descriptors3D.NPR1(mol),
        'NPR2': Descriptors3D.NPR2(mol),
    }

    return descriptors

# SMILES에 대해 3D 서술자 계산
descriptors_list = []
for smiles in df['Smiles']:
    descriptors = compute_3d_descriptors(smiles)
    descriptors_list.append(descriptors)

# 계산된 서술자를 데이터프레임으로 변환
descriptors_df = pd.DataFrame(descriptors_list)

# 원래의 train 데이터프레임에 추가
df_3d = pd.concat([df, descriptors_df], axis=1)

In [25]:
df_3d

,Smiles,IC50_nM,pIC50,Exact Mass,XLogP,Hydrogen Bond Donors,Hydrogen Bond Acceptors,Rotatable Bonds,TPSA,Charge,...,Heteroatom Count by RDkit,BalabanJ Index by RDkit,Asphericity,Eccentricity,SpherocityIndex,PMI1,PMI2,PMI3,NPR1,NPR2
0,[H]c1nc(OC([H])([H])[C@@]2([H])N([H])C(=O)C([H...,0.022,10.66,994.51640508,2.8,6,13,24,251.0,0,...,19,0.858127,0.723796,0.994683,0.104516,9315.136892,86481.531267,90455.343033,0.102981,0.956069
1,[H]OC(C([H])([H])[H])(C([H])([H])[H])[C@]([H])...,0.026,10.59,535.27071613,1.4,2,8,7,106.0,0,...,11,1.295765,0.436704,0.969510,0.162385,3680.565802,12782.147170,15019.452406,0.245053,0.851039
2,[H]OC(C([H])([H])[H])(C([H])([H])[H])[C@]([H])...,0.078,10.11,537.24998069,0.2,2,9,7,116.0,0,...,12,1.295765,0.336248,0.948919,0.142041,4975.671926,11954.565173,15769.764779,0.315520,0.758069
3,[H]OC(C([H])([H])[H])(C([H])([H])[H])[C@]([H])...,0.081,10.09,545.23622233,1.3,2,10,8,106.0,0,...,13,1.446176,0.300943,0.936442,0.093742,6088.435964,12164.886082,17354.761416,0.350822,0.700954
4,[H]O[C@@]1([H])C([H])([H])N(C(=O)[C@@]([H])(N(...,0.099,10.00,935.46153323,7.5,5,11,23,243.0,0,...,16,0.894084,0.758419,0.996025,0.122920,7553.856255,82857.219728,84799.549124,0.089079,0.977095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,[H]c1c([H])c(C(=O)N([H])c2nc3c([H])c([H])n([H]...,30000.000,4.52,283.07053917,1.4,2,5,2,117.0,0,...,8,1.847029,0.672488,0.992224,0.042621,599.848952,4321.316841,4819.518158,0.124462,0.896628
1948,[H]c1c([H])c(Cl)c([H])c(C(=O)N([H])c2nc3c([H])...,30000.000,4.52,327.1138399,4.4,1,2,5,46.9,0,...,5,1.935112,0.381605,0.959938,0.150184,1501.824827,4412.768847,5359.587375,0.280213,0.823341
1949,[H]c1c([H])c(C(=O)N([H])c2nc3c([H])c(F)c(F)c([...,30000.000,4.52,318.05644645,2.7,2,6,2,104.0,0,...,9,1.877989,0.797941,0.997355,0.066654,505.093785,6633.751103,6948.510034,0.072691,0.954701
1950,[H]OC([H])([H])[C@@]1([H])C([H])([H])[C@@]([H]...,42000.000,4.38,449.15216079,3.6,5,9,6,152.0,0,...,9,1.516899,0.254053,0.911832,0.102687,4040.534299,6243.315846,9841.442533,0.410563,0.634390


# 4. 기존 IRAK4 억제제와의 유사성 변수 추출

In [26]:
def get_3d_structure(smiles):
    """ Convert SMILES to 3D structure and return the molecule """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.UFFOptimizeMolecule(mol)
    return mol
    
def calculate_similarity(mol1, mol2):
    """ Calculate Tanimoto similarity between two molecules based on their fingerprints """
    if mol1 is None or mol2 is None:
        return np.nan
    
    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=2048)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 2, nBits=2048)
    return DataStructs.TanimotoSimilarity(fp1, fp2)

# Define the reference SMILES
reference_smiles = "COC1=C(C=CC(=C1)N2CCOCC2)NC(=O)C3=CC=CC(=N3)C4=CC=NN4"
reference_mol = get_3d_structure(reference_smiles)

# Calculate similarity and add it to the DataFrame
def add_similarity_column(df, reference_smiles):
    reference_mol = get_3d_structure(reference_smiles)
    similarities = []
    for smiles in df['Smiles']:
        mol = get_3d_structure(smiles)
        similarity = calculate_similarity(reference_mol, mol)
        similarities.append(similarity)
    df['Similarity'] = similarities


# Add similarity column
add_similarity_column(df_3d, reference_smiles)

[04:33:20] DEPRECATION WARNING: please use MorganGenerator
[04:33:20] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:21] DEPRECATION WARNING: please use MorganGenerator
[04:33:22] DEPRECATION WARNING: please use MorganGenerat

In [27]:
df_3d

,Smiles,IC50_nM,pIC50,Exact Mass,XLogP,Hydrogen Bond Donors,Hydrogen Bond Acceptors,Rotatable Bonds,TPSA,Charge,...,BalabanJ Index by RDkit,Asphericity,Eccentricity,SpherocityIndex,PMI1,PMI2,PMI3,NPR1,NPR2,Similarity
0,[H]c1nc(OC([H])([H])[C@@]2([H])N([H])C(=O)C([H...,0.022,10.66,994.51640508,2.8,6,13,24,251.0,0,...,0.858127,0.723796,0.994683,0.104516,9315.136892,86481.531267,90455.343033,0.102981,0.956069,0.154762
1,[H]OC(C([H])([H])[H])(C([H])([H])[H])[C@]([H])...,0.026,10.59,535.27071613,1.4,2,8,7,106.0,0,...,1.295765,0.436704,0.969510,0.162385,3680.565802,12782.147170,15019.452406,0.245053,0.851039,0.198413
2,[H]OC(C([H])([H])[H])(C([H])([H])[H])[C@]([H])...,0.078,10.11,537.24998069,0.2,2,9,7,116.0,0,...,1.295765,0.336248,0.948919,0.142041,4975.671926,11954.565173,15769.764779,0.315520,0.758069,0.216000
3,[H]OC(C([H])([H])[H])(C([H])([H])[H])[C@]([H])...,0.081,10.09,545.23622233,1.3,2,10,8,106.0,0,...,1.446176,0.300943,0.936442,0.093742,6088.435964,12164.886082,17354.761416,0.350822,0.700954,0.208333
4,[H]O[C@@]1([H])C([H])([H])N(C(=O)[C@@]([H])(N(...,0.099,10.00,935.46153323,7.5,5,11,23,243.0,0,...,0.894084,0.758419,0.996025,0.122920,7553.856255,82857.219728,84799.549124,0.089079,0.977095,0.157576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,[H]c1c([H])c(C(=O)N([H])c2nc3c([H])c([H])n([H]...,30000.000,4.52,283.07053917,1.4,2,5,2,117.0,0,...,1.847029,0.672488,0.992224,0.042621,599.848952,4321.316841,4819.518158,0.124462,0.896628,0.208791
1948,[H]c1c([H])c(Cl)c([H])c(C(=O)N([H])c2nc3c([H])...,30000.000,4.52,327.1138399,4.4,1,2,5,46.9,0,...,1.935112,0.381605,0.959938,0.150184,1501.824827,4412.768847,5359.587375,0.280213,0.823341,0.221053
1949,[H]c1c([H])c(C(=O)N([H])c2nc3c([H])c(F)c(F)c([...,30000.000,4.52,318.05644645,2.7,2,6,2,104.0,0,...,1.877989,0.797941,0.997355,0.066654,505.093785,6633.751103,6948.510034,0.072691,0.954701,0.206522
1950,[H]OC([H])([H])[C@@]1([H])C([H])([H])[C@@]([H]...,42000.000,4.38,449.15216079,3.6,5,9,6,152.0,0,...,1.516899,0.254053,0.911832,0.102687,4040.534299,6243.315846,9841.442533,0.410563,0.634390,0.123894


# 5. 불필요 변수 제거/마무리

In [28]:
# 'Covalently-Bonded Unit Count by RDkit' 및 'Compound Complexity (Morgan) by RDkit' 열을 제거
df_3d_filtered = df_3d.drop(columns=['Covalently-Bonded Unit Count by RDkit', 'Compound Complexity (Morgan) by RDkit'])

# 최종 데이터프레임 이름을 vector_3d_sim_train으로 지정
vector_3d_sim_train = df_3d_filtered

In [29]:
# CSV 파일로 내보내기
vector_3d_sim_train.to_csv('vector_3d_sim_train.csv', index=False)

# ----------------------------------------------------------------------------------

# 0. test data 불러오기

In [30]:
ch_df_test=pd.read_csv('test.csv')

In [31]:
ch_df_test

,ID,Smiles
0,TEST_000,O=C(C1=CSC(C2=CC=CN=C2)=N1)NC3=CC(NC4CCN(C)CC4...
1,TEST_001,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(NC4CCCCC4)=...
2,TEST_002,N#CC(C=C1)=C(N[C@@H]2CCNC2)C=C1NC(N=C3)=NC=C3C...
3,TEST_003,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(NC(C=C)=O)CC...
4,TEST_004,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(N)CC3)=C2C(C...
...,...,...
108,TEST_108,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(N4CCOCC4)=C...
109,TEST_109,O=C(C1=CSC(C2=CC=NC=C2)=N1)NC3=CC(NC4CCN(C(C)C...
110,TEST_110,N#Cc1ccc(Nc2ncc(cn2)c3cnn(c3)C4CCNCC4)cc1N[C@@...
111,TEST_111,O=C(C)N(CC1)CCC1N2N=CC(C3=CN=C(N4C(C=CC(C#N)=C...


# 1~2. Rdkit계산으로 추출한 변수(test)

In [32]:


# 특성을 저장할 빈 리스트 생성
features = []

# 각 SMILES 문자열에 대해 RDKit으로 특성을 계산
for smiles in ch_df_test['Smiles']:
    molecule = Chem.MolFromSmiles(smiles)
    if molecule:
        # MorganGenerator를 사용하여 복잡성 계산
        morgan_gen = GetMorganGenerator(radius=2)
        morgan_fp = morgan_gen.GetFingerprint(molecule)
        
        # 특성 계산
        info = {
            "Exact Mass": Descriptors.ExactMolWt(molecule),
            "XLogP": Crippen.MolLogP(molecule),
            "Hydrogen Bond Donors": Lipinski.NumHDonors(molecule),
            "Hydrogen Bond Acceptors": Lipinski.NumHAcceptors(molecule),
            "Rotatable Bonds": Descriptors.NumRotatableBonds(molecule),
            "TPSA": rdMolDescriptors.CalcTPSA(molecule),
            "Charge": Chem.GetFormalCharge(molecule),
            "Heavy Atom Count": Descriptors.HeavyAtomCount(molecule),
            "Covalently-Bonded Unit Count": len(Chem.GetMolFrags(molecule)),
        }
    else:
        # 만약 SMILES가 유효하지 않은 경우, NaN 값으로 채움
        info = {key: float('nan') for key in [
            "Exact Mass", "XLogP", "Hydrogen Bond Donors", 
            "Hydrogen Bond Acceptors", "Rotatable Bonds", 
            "TPSA", "Charge", "Heavy Atom Count", 
            "Covalently-Bonded Unit Count"
        ]}
    
    features.append(info)

# 특성을 DataFrame으로 변환하고 원본 DataFrame과 병합
features_df = pd.DataFrame(features)
result_df_test = pd.concat([ch_df_test, features_df], axis=1)

In [33]:
result_df_test

,ID,Smiles,Exact Mass,XLogP,Hydrogen Bond Donors,Hydrogen Bond Acceptors,Rotatable Bonds,TPSA,Charge,Heavy Atom Count,Covalently-Bonded Unit Count
0,TEST_000,O=C(C1=CSC(C2=CC=CN=C2)=N1)NC3=CC(NC4CCN(C)CC4...,477.219846,5.28490,2,7,7,79.38,0,34,1
1,TEST_001,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(NC4CCCCC4)=...,466.259343,4.82478,2,8,5,96.38,0,35,1
2,TEST_002,N#CC(C=C1)=C(N[C@@H]2CCNC2)C=C1NC(N=C3)=NC=C3C...,415.223292,2.26358,4,9,6,115.51,0,31,1
3,TEST_003,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(NC(C=C)=O)CC...,498.249172,3.94208,3,9,8,129.78,0,37,1
4,TEST_004,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(N)CC3)=C2C(C...,444.238608,3.59858,3,9,6,126.70,0,33,1
...,...,...,...,...,...,...,...,...,...,...,...
108,TEST_108,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(N4CCOCC4)=C...,454.222957,2.91678,1,9,4,96.82,0,34,1
109,TEST_109,O=C(C1=CSC(C2=CC=NC=C2)=N1)NC3=CC(NC4CCN(C(C)C...,561.288594,4.84700,3,8,8,102.49,0,40,1
110,TEST_110,N#Cc1ccc(Nc2ncc(cn2)c3cnn(c3)C4CCNCC4)cc1N[C@@...,429.238942,2.65368,4,9,6,115.51,0,32,1
111,TEST_111,O=C(C)N(CC1)CCC1N2N=CC(C3=CN=C(N4C(C=CC(C#N)=C...,411.180758,3.33908,0,7,3,92.63,0,31,1


In [34]:
#RDkit로 추가할 수 있는 feature들 추가.


# 함수: 분자식으로부터 다양한 지표를 계산하는 함수
def calculate_descriptors(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return None
    
    # Morgan Fingerprint 생성기 설정
    morgan_gen = GetMorganGenerator(radius=2, fpSize=2048)
    
    # 지표 계산
    descriptors = {
        'Smiles': smiles,
        'Complexity by RDkit': Descriptors.BertzCT(molecule),
        'Covalently-Bonded Unit Count by RDkit': len(Chem.GetMolFrags(molecule)),
        'Compound Complexity (Morgan) by RDkit': morgan_gen.GetFingerprint(molecule).GetNumOnBits(),
        'Fraction of sp3 Carbons by RDkit': Descriptors.FractionCSP3(molecule),
        'Ring Count by RDkit': Descriptors.RingCount(molecule),
        'Aromatic Ring Count by RDkit': Descriptors.NumAromaticRings(molecule),
        'Heteroatom Count by RDkit': Descriptors.NumHeteroatoms(molecule),
        'BalabanJ Index by RDkit': Descriptors.BalabanJ(molecule),
    }
    
    return descriptors

# 데이터프레임에 지표 계산 적용
def create_descriptors_dataframe(ch_df_test):
    calculated_data = []
    for smiles in ch_df_test['Smiles']:
        descriptors = calculate_descriptors(smiles)
        if descriptors:
            calculated_data.append(descriptors)
    
    return pd.DataFrame(calculated_data)

# ch_df_test 데이터프레임을 불러온 후 사용
rdkit_df_test = create_descriptors_dataframe(ch_df_test)
rdkit_df_test

,Smiles,Complexity by RDkit,Covalently-Bonded Unit Count by RDkit,Compound Complexity (Morgan) by RDkit,Fraction of sp3 Carbons by RDkit,Ring Count by RDkit,Aromatic Ring Count by RDkit,Heteroatom Count by RDkit,BalabanJ Index by RDkit
0,O=C(C1=CSC(C2=CC=CN=C2)=N1)NC3=CC(NC4CCN(C)CC4...,1105.388583,1,67,0.423077,5,3,8,1.337668
1,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(NC4CCCCC4)=...,1365.593121,1,64,0.407407,6,4,8,1.376728
2,N#CC(C=C1)=C(N[C@@H]2CCNC2)C=C1NC(N=C3)=NC=C3C...,1071.106189,1,58,0.363636,5,3,9,1.285447
3,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(NC(C=C)=O)CC...,1290.480108,1,70,0.370370,5,3,10,1.386093
4,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(N)CC3)=C2C(C...,1130.111141,1,63,0.416667,5,3,9,1.417649
...,...,...,...,...,...,...,...,...,...
108,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(N4CCOCC4)=C...,1355.003353,1,64,0.360000,6,4,9,1.409469
109,O=C(C1=CSC(C2=CC=NC=C2)=N1)NC3=CC(NC4CCN(C(C)C...,1296.395880,1,66,0.466667,5,3,10,1.330197
110,N#Cc1ccc(Nc2ncc(cn2)c3cnn(c3)C4CCNCC4)cc1N[C@@...,1087.400007,1,58,0.391304,5,3,9,1.274013
111,O=C(C)N(CC1)CCC1N2N=CC(C3=CN=C(N4C(C=CC(C#N)=C...,1287.711257,1,58,0.260870,5,4,8,1.344626


In [35]:
# rdkit_df_test에서 'Smiles' 칼럼을 제거
rdkit_df_test_without_smiles = rdkit_df_test.drop(columns=['Smiles'])

# 두 데이터프레임을 이어 붙임 (열을 기준으로 합침)
df_test = pd.concat([result_df_test, rdkit_df_test_without_smiles], axis=1)
df_test

,ID,Smiles,Exact Mass,XLogP,Hydrogen Bond Donors,Hydrogen Bond Acceptors,Rotatable Bonds,TPSA,Charge,Heavy Atom Count,Covalently-Bonded Unit Count,Complexity by RDkit,Covalently-Bonded Unit Count by RDkit,Compound Complexity (Morgan) by RDkit,Fraction of sp3 Carbons by RDkit,Ring Count by RDkit,Aromatic Ring Count by RDkit,Heteroatom Count by RDkit,BalabanJ Index by RDkit
0,TEST_000,O=C(C1=CSC(C2=CC=CN=C2)=N1)NC3=CC(NC4CCN(C)CC4...,477.219846,5.28490,2,7,7,79.38,0,34,1,1105.388583,1,67,0.423077,5,3,8,1.337668
1,TEST_001,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(NC4CCCCC4)=...,466.259343,4.82478,2,8,5,96.38,0,35,1,1365.593121,1,64,0.407407,6,4,8,1.376728
2,TEST_002,N#CC(C=C1)=C(N[C@@H]2CCNC2)C=C1NC(N=C3)=NC=C3C...,415.223292,2.26358,4,9,6,115.51,0,31,1,1071.106189,1,58,0.363636,5,3,9,1.285447
3,TEST_003,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(NC(C=C)=O)CC...,498.249172,3.94208,3,9,8,129.78,0,37,1,1290.480108,1,70,0.370370,5,3,10,1.386093
4,TEST_004,N#CC(C=C1)=CC=C1NC(N=C2)=NC(NC3CC(N)CC3)=C2C(C...,444.238608,3.59858,3,9,6,126.70,0,33,1,1130.111141,1,63,0.416667,5,3,9,1.417649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,TEST_108,N#CC1=CC(C=C2)=C(C=C1)N2C(N=C3)=NC(N4CCOCC4)=C...,454.222957,2.91678,1,9,4,96.82,0,34,1,1355.003353,1,64,0.360000,6,4,9,1.409469
109,TEST_109,O=C(C1=CSC(C2=CC=NC=C2)=N1)NC3=CC(NC4CCN(C(C)C...,561.288594,4.84700,3,8,8,102.49,0,40,1,1296.395880,1,66,0.466667,5,3,10,1.330197
110,TEST_110,N#Cc1ccc(Nc2ncc(cn2)c3cnn(c3)C4CCNCC4)cc1N[C@@...,429.238942,2.65368,4,9,6,115.51,0,32,1,1087.400007,1,58,0.391304,5,3,9,1.274013
111,TEST_111,O=C(C)N(CC1)CCC1N2N=CC(C3=CN=C(N4C(C=CC(C#N)=C...,411.180758,3.33908,0,7,3,92.63,0,31,1,1287.711257,1,58,0.260870,5,4,8,1.344626


# 3. 여러가지 3D 구조열 변수 추출(test)

In [36]:


# SMILES 문자열에 수소 원자를 추가하는 함수
def add_hydrogens(smiles):
    mol = Chem.MolFromSmiles(smiles)  # SMILES 문자열을 Mol 객체로 변환
    mol_with_hs = Chem.AddHs(mol)     # 수소 원자 추가
    return Chem.MolToSmiles(mol_with_hs)  # Mol 객체를 SMILES 문자열로 변환

# df 데이터프레임에 수소 원자를 추가한 SMILES 열 생성
df_test['Smiles'] = df_test['Smiles'].apply(add_hydrogens)

In [37]:
import numpy as np
import random
# 랜덤 시드 고정
seed = 42
random.seed(seed)
np.random.seed(seed)

# 3D 서술자를 계산하는 함수 정의
def compute_3d_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # 수소 추가
    
    # 시드를 고정하여 3D 구조 생성
    AllChem.EmbedMolecule(mol, randomSeed=seed)  # 3D 구조 생성 시 시드 적용
    AllChem.UFFOptimizeMolecule(mol)  # 구조 최적화

    # 3D 서술자 계산
    descriptors = {
        'Asphericity': Descriptors3D.Asphericity(mol),
        'Eccentricity': Descriptors3D.Eccentricity(mol),
        'SpherocityIndex': Descriptors3D.SpherocityIndex(mol),
        'PMI1': Descriptors3D.PMI1(mol),
        'PMI2': Descriptors3D.PMI2(mol),
        'PMI3': Descriptors3D.PMI3(mol),
        'NPR1': Descriptors3D.NPR1(mol),
        'NPR2': Descriptors3D.NPR2(mol),
    }

    return descriptors

# SMILES에 대해 3D 서술자 계산
descriptors_list = []
for smiles in df_test['Smiles']:
    descriptors = compute_3d_descriptors(smiles)
    descriptors_list.append(descriptors)

# 계산된 서술자를 데이터프레임으로 변환
descriptors_df = pd.DataFrame(descriptors_list)

# 원래의 train 데이터프레임에 추가
df_3d_test = pd.concat([df_test, descriptors_df], axis=1)

In [38]:
df_3d_test

,ID,Smiles,Exact Mass,XLogP,Hydrogen Bond Donors,Hydrogen Bond Acceptors,Rotatable Bonds,TPSA,Charge,Heavy Atom Count,...,Heteroatom Count by RDkit,BalabanJ Index by RDkit,Asphericity,Eccentricity,SpherocityIndex,PMI1,PMI2,PMI3,NPR1,NPR2
0,TEST_000,[H]c1nc([H])c(-c2nc(C(=O)N([H])c3c([H])c(N([H]...,477.219846,5.28490,2,7,7,79.38,0,34,...,8,1.337668,0.412741,0.965216,0.103963,3676.983979,11138.105971,14063.597590,0.261454,0.791981
1,TEST_001,[H]c1nc(-n2c([H])c([H])c3c([H])c(C#N)c([H])c([...,466.259343,4.82478,2,8,5,96.38,0,35,...,8,1.376728,0.440997,0.970159,0.175914,2934.942675,10328.871764,12104.338344,0.242470,0.853320
2,TEST_002,[H]c1nc(N([H])c2c([H])c([H])c(C#N)c(N([H])[C@@...,415.223292,2.26358,4,9,6,115.51,0,31,...,9,1.285447,0.865381,0.998885,0.062992,720.622553,14929.019442,15264.475457,0.047209,0.978024
3,TEST_003,[H]C([H])=C([H])C(=O)N([H])C1([H])C([H])([H])C...,498.249172,3.94208,3,9,8,129.78,0,37,...,10,1.386093,0.226872,0.868989,0.091343,8079.420045,8806.184125,16327.640649,0.494831,0.539342
4,TEST_004,[H]c1nc(N([H])c2c([H])c([H])c(C#N)c([H])c2[H])...,444.238608,3.59858,3,9,6,126.70,0,33,...,9,1.417649,0.422879,0.967059,0.135951,3136.386197,9906.259570,12321.215712,0.254552,0.804000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,TEST_108,[H]c1nc(-n2c([H])c([H])c3c([H])c(C#N)c([H])c([...,454.222957,2.91678,1,9,4,96.82,0,34,...,9,1.409469,0.418038,0.965882,0.107427,3152.311320,9492.881247,12171.842358,0.258984,0.779905
109,TEST_109,[H]c1nc([H])c([H])c(-c2nc(C(=O)N([H])c3c([H])c...,561.288594,4.84700,3,8,8,102.49,0,40,...,10,1.330197,0.302634,0.936436,0.097614,7399.668824,14641.263332,21091.453932,0.350837,0.694180
110,TEST_110,[H]c1nc(N([H])c2c([H])c([H])c(C#N)c(N([H])[C@@...,429.238942,2.65368,4,9,6,115.51,0,32,...,9,1.274013,0.813021,0.997763,0.062987,1108.633215,15883.983321,16585.451150,0.066844,0.957706
111,TEST_111,[H]c1nc(-n2c([H])c([H])c3c([H])c(C#N)c([H])c([...,411.180758,3.33908,0,7,3,92.63,0,31,...,8,1.344626,0.808291,0.997644,0.054552,976.360617,13492.554162,14231.553745,0.068605,0.948073


# 4. 기존 IRAK4 억제제와의 유사성 변수 추출(test)

In [39]:
from rdkit.Chem import DataStructs
def get_3d_structure(smiles):
    """ Convert SMILES to 3D structure and return the molecule """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.UFFOptimizeMolecule(mol)
    return mol
    
def calculate_similarity(mol1, mol2):
    """ Calculate Tanimoto similarity between two molecules based on their fingerprints """
    if mol1 is None or mol2 is None:
        return np.nan
    
    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=2048)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 2, nBits=2048)
    return DataStructs.TanimotoSimilarity(fp1, fp2)

# Define the reference SMILES
reference_smiles = "COC1=C(C=CC(=C1)N2CCOCC2)NC(=O)C3=CC=CC(=N3)C4=CC=NN4"
reference_mol = get_3d_structure(reference_smiles)

# Calculate similarity and add it to the DataFrame
def add_similarity_column(df, reference_smiles):
    reference_mol = get_3d_structure(reference_smiles)
    similarities = []
    for smiles in df['Smiles']:
        mol = get_3d_structure(smiles)
        similarity = calculate_similarity(reference_mol, mol)
        similarities.append(similarity)
    df['Similarity'] = similarities


# Add similarity column
add_similarity_column(df_3d_test, reference_smiles)

[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerator
[04:35:19] DEPRECATION WARNING: please use MorganGenerat

In [40]:
df_3d_test

,ID,Smiles,Exact Mass,XLogP,Hydrogen Bond Donors,Hydrogen Bond Acceptors,Rotatable Bonds,TPSA,Charge,Heavy Atom Count,...,BalabanJ Index by RDkit,Asphericity,Eccentricity,SpherocityIndex,PMI1,PMI2,PMI3,NPR1,NPR2,Similarity
0,TEST_000,[H]c1nc([H])c(-c2nc(C(=O)N([H])c3c([H])c(N([H]...,477.219846,5.28490,2,7,7,79.38,0,34,...,1.337668,0.412741,0.965216,0.103963,3676.983979,11138.105971,14063.597590,0.261454,0.791981,0.261682
1,TEST_001,[H]c1nc(-n2c([H])c([H])c3c([H])c(C#N)c([H])c([...,466.259343,4.82478,2,8,5,96.38,0,35,...,1.376728,0.440997,0.970159,0.175914,2934.942675,10328.871764,12104.338344,0.242470,0.853320,0.126050
2,TEST_002,[H]c1nc(N([H])c2c([H])c([H])c(C#N)c(N([H])[C@@...,415.223292,2.26358,4,9,6,115.51,0,31,...,1.285447,0.865381,0.998885,0.062992,720.622553,14929.019442,15264.475457,0.047209,0.978024,0.154545
3,TEST_003,[H]C([H])=C([H])C(=O)N([H])C1([H])C([H])([H])C...,498.249172,3.94208,3,9,8,129.78,0,37,...,1.386093,0.226872,0.868989,0.091343,8079.420045,8806.184125,16327.640649,0.494831,0.539342,0.158333
4,TEST_004,[H]c1nc(N([H])c2c([H])c([H])c(C#N)c([H])c2[H])...,444.238608,3.59858,3,9,6,126.70,0,33,...,1.417649,0.422879,0.967059,0.135951,3136.386197,9906.259570,12321.215712,0.254552,0.804000,0.137931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,TEST_108,[H]c1nc(-n2c([H])c([H])c3c([H])c(C#N)c([H])c([...,454.222957,2.91678,1,9,4,96.82,0,34,...,1.409469,0.418038,0.965882,0.107427,3152.311320,9492.881247,12171.842358,0.258984,0.779905,0.187500
109,TEST_109,[H]c1nc([H])c([H])c(-c2nc(C(=O)N([H])c3c([H])c...,561.288594,4.84700,3,8,8,102.49,0,40,...,1.330197,0.302634,0.936436,0.097614,7399.668824,14641.263332,21091.453932,0.350837,0.694180,0.241071
110,TEST_110,[H]c1nc(N([H])c2c([H])c([H])c(C#N)c(N([H])[C@@...,429.238942,2.65368,4,9,6,115.51,0,32,...,1.274013,0.813021,0.997763,0.062987,1108.633215,15883.983321,16585.451150,0.066844,0.957706,0.154545
111,TEST_111,[H]c1nc(-n2c([H])c([H])c3c([H])c(C#N)c([H])c([...,411.180758,3.33908,0,7,3,92.63,0,31,...,1.344626,0.808291,0.997644,0.054552,976.360617,13492.554162,14231.553745,0.068605,0.948073,0.165138


# 5. 불필요 변수 제거/마무리(test)

In [41]:
# 'Covalently-Bonded Unit Count by RDkit' 및 'Compound Complexity (Morgan) by RDkit' 열을 제거
df_3d_test_filtered = df_3d_test.drop(columns=['Covalently-Bonded Unit Count by RDkit', 'Compound Complexity (Morgan) by RDkit'])

# 최종 데이터프레임 이름을 vector_3d_sim_train으로 지정
vector_3d_sim_test = df_3d_test_filtered

In [42]:
# CSV 파일로 내보내기
vector_3d_sim_test.to_csv('vector_3d_sim_test.csv', index=False)

# Mpnn stacking(앙상블)

In [43]:
import joblib
import random 
import pandas as pd
import numpy as np
from rdkit import Chem
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import add_self_loops

# Seed 고정 함수
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Seed 설정
seed = 29238
seed_everything(seed)

# Atom feature 함수
def get_atom_features(atom):
    atom_type = atom.GetAtomicNum()
    degree = atom.GetDegree()
    formal_charge = atom.GetFormalCharge()
    hybridization = int(atom.GetHybridization())
    is_aromatic = int(atom.GetIsAromatic())
    chirality = int(atom.GetChiralTag())
    num_hs = atom.GetTotalNumHs()
    
    return [atom_type, degree, formal_charge, hybridization, is_aromatic, chirality, num_hs]

# Bond feature 함수
def get_bond_features(bond):
    bond_type_dict = {
        Chem.rdchem.BondType.SINGLE: 0,
        Chem.rdchem.BondType.DOUBLE: 1,
        Chem.rdchem.BondType.TRIPLE: 2,
        Chem.rdchem.BondType.AROMATIC: 3
    }
    bond_type_idx = bond_type_dict.get(bond.GetBondType(), 4)
    return [bond_type_idx, int(bond.GetIsConjugated()), int(bond.IsInRing()), int(bond.GetStereo())]

# MoleculeDataset 클래스
class MoleculeDataset(Dataset):
    def __init__(self, dataframe, is_train=True):
        self.dataframe = dataframe.reset_index(drop=True)
        self.is_train = is_train
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        smiles = row['Smiles']
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        
        data = self.mol_to_graph_data_obj_simple(mol)
        if self.is_train:
            target = row['pIC50']
            data.y = torch.tensor([target], dtype=torch.float)
        return data
    
    def mol_to_graph_data_obj_simple(self, mol):
        atom_features_list = [get_atom_features(atom) for atom in mol.GetAtoms()]
        x = torch.tensor(atom_features_list, dtype=torch.float)
        
        edge_index = []
        edge_attr = []
        for bond in mol.GetBonds():
            i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            edge_index.append([i, j])
            edge_index.append([j, i])
            bond_feature = get_bond_features(bond)
            edge_attr.append(bond_feature)
            edge_attr.append(bond_feature)
        
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous() if len(edge_index) > 0 else torch.empty((2, 0), dtype=torch.long)
        edge_attr = torch.tensor(edge_attr, dtype=torch.float) if len(edge_attr) > 0 else torch.empty((0, 4), dtype=torch.float)
        
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# MPNNLayer 클래스 정의
class MPNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(MPNNLayer, self).__init__(aggr='add')
        self.mlp = nn.Sequential(
            nn.Linear(in_channels * 2, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels)
        )
        
    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        return self.propagate(edge_index, x=x)
    
    def message(self, x_i, x_j):
        return self.mlp(torch.cat([x_i, x_j], dim=1))
    
    def update(self, aggr_out):
        return aggr_out

# MPNN 모델 정의
class MPNNModel(torch.nn.Module):
    def __init__(self, num_node_features):
        super(MPNNModel, self).__init__()
        self.mpnn1 = MPNNLayer(num_node_features, 64)
        self.mpnn2 = MPNNLayer(64, 128)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.mpnn1(x, edge_index))
        x = F.relu(self.mpnn2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# 손실 계산
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)
        out = model(data)
        loss = criterion(out.view(-1), data.y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

# 모델 학습 및 seed 반복 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_df = pd.read_csv('vector_3d_sim_train.csv')
test_df = pd.read_csv('vector_3d_sim_test.csv')

train_dataset = MoleculeDataset(train_df, is_train=True)
test_dataset = MoleculeDataset(test_df, is_train=False)

train_dataset = [data for data in train_dataset if data is not None]
test_dataset = [data for data in test_dataset if data is not None]

batch_size = 32
train_loader = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # 학습용 로더
test_loader = PyGDataLoader(test_dataset, batch_size=batch_size, shuffle=False)   # 테스트용 로더

criterion = nn.MSELoss()

# 모델 정의 및 학습
model = MPNNModel(num_node_features=7).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
for epoch in range(1, 400):
    loss = train(model, train_loader, optimizer, criterion)
    print(f'Epoch {epoch}, Loss: {loss}')
    
# 테스트 데이터에 대한 예측 및 결과 저장
model.eval()
predictions = []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out = model(data)
        predictions.extend(out.cpu().numpy())
    
test_y_pred = np.array(predictions).flatten()

# pIC50 값을 IC50_nM으로 변환하는 함수
def pIC50_to_IC50(pic50_values):
    return 10 ** (9 - pic50_values)
    
predicted_IC50_nM = pIC50_to_IC50(test_y_pred)

test_df['mpnn_predict'] = predicted_IC50_nM

# CSV 파일 저장
test_df.to_csv('final_test.csv', index=False)

# 예측용 훈련 데이터 로더 생성 (shuffle=False)
train_loader_inference = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# 훈련 데이터에 대한 예측 수행 및 결과 저장
train_predictions = []
with torch.no_grad():
    for data in train_loader_inference:
        data = data.to(device)
        out = model(data)
        train_predictions.extend(out.cpu().numpy())

train_y_pred = np.array(train_predictions).flatten()

predicted_train_IC50_nM = pIC50_to_IC50(train_y_pred)

train_df['mpnn_predict'] = predicted_train_IC50_nM

# CSV 파일 저장
train_df.to_csv('final_train.csv', index=False)



Epoch 1, Loss: 3.8545506664964018
Epoch 2, Loss: 1.340236623756221
Epoch 3, Loss: 1.334882242757766
Epoch 4, Loss: 1.2346657084637
Epoch 5, Loss: 1.2267952756803544
Epoch 6, Loss: 1.249468743801117
Epoch 7, Loss: 1.146540072120604
Epoch 8, Loss: 1.116956356118937
Epoch 9, Loss: 1.0033016072922065
Epoch 10, Loss: 0.957760440521553
Epoch 11, Loss: 0.9136736979250049
Epoch 12, Loss: 0.9403955643294287
Epoch 13, Loss: 0.956649769525059
Epoch 14, Loss: 0.9352128295624842
Epoch 15, Loss: 1.0528549278368715
Epoch 16, Loss: 0.9914077158834114
Epoch 17, Loss: 0.9104149282955732
Epoch 18, Loss: 0.8554069096924829
Epoch 19, Loss: 0.8902432425100295
Epoch 20, Loss: 0.8813943174041685
Epoch 21, Loss: 0.826547022725715
Epoch 22, Loss: 0.9164198773806213
Epoch 23, Loss: 0.8394857439838472
Epoch 24, Loss: 0.8479772801281976
Epoch 25, Loss: 0.8829608115016437
Epoch 26, Loss: 0.7983744144439697
Epoch 27, Loss: 0.8862734832724587
Epoch 28, Loss: 0.7861733675980177
Epoch 29, Loss: 0.7914198925260638
Epoch

In [54]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('final_train.csv')
df2 = pd.read_csv('final_test.csv')
docking_score_train = pd.read_csv('docking_score_train.csv')
docking_score_test = pd.read_csv('docking_score_test.csv')

# 행 개수를 기준으로 오른쪽으로 병합
df = pd.concat([df, docking_score_train], axis=1)
df2 = pd.concat([df2, docking_score_test], axis=1)

# 결과를 새로운 CSV 파일로 저장
df.to_csv('final_train+docking_score_train.csv', index=False)
df2.to_csv('final_test+docking_score_test.csv', index=False)

print("CSV 파일이 성공적으로 저장되었습니다.")


CSV 파일이 성공적으로 저장되었습니다.


In [5]:
df = pd.read_csv('final_train+docking_score_train.csv')
df2 = pd.read_csv('final_test+docking_score_test.csv')

In [6]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr
import joblib
import pandas as pd

# 데이터 준비
numeric_df = df.select_dtypes(include=[np.number])  
numeric_df = numeric_df.dropna()
y = numeric_df['pIC50']
X = numeric_df.drop(columns=['IC50_nM', 'pIC50'], errors='ignore')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 직접 하이퍼파라미터 설정
manual_params = {
    'max_depth': 3,           
    'n_estimators': 185,     
    'learning_rate': 0.02515906570522222,   
    'subsample':  0.6084076093673286,      
    'colsample_bytree':  0.98235388905852  
}

# 모델 학습 및 평가
model = xgb.XGBRegressor(
    max_depth=manual_params['max_depth'],
    n_estimators=manual_params['n_estimators'],
    learning_rate=manual_params['learning_rate'],
    subsample=manual_params['subsample'],
    colsample_bytree=manual_params['colsample_bytree'],
    random_state=42
)

# 모델 학습 시 가중치 적용하지 않음
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# IC50_nM 단위의 Normalized RMSE 오차 계산 (A)
y_test_nM = 10 ** (-y_test + 9)
y_pred_nM = 10 ** (-y_pred + 9)

mse = mean_squared_error(y_test_nM, y_pred_nM)
rmse = np.sqrt(mse)
A = rmse / (np.max(y_test_nM) - np.min(y_test_nM))

# pIC50 단위로 변환하여 Correct Ratio (Absolute Error <= 0.5) 계산 (B)
correct_ratio = np.mean(np.abs(y_test - y_pred) <= 0.5)
B = correct_ratio

# 최종 Score 계산
Score = 0.5 * (1 - min(A, 1)) + 0.5 * B

# R^2 계산
r2 = r2_score(y_test, y_pred)

# Adjusted R^2 계산
n = X_test.shape[0]  # number of observations
p = X_test.shape[1]  # number of predictors
adjusted_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

# Pearson 상관계수 계산
pearson_corr, _ = pearsonr(y_test, y_pred)

# 모델 저장 (간결한 모델 이름)
model_name = "XGBOOST_IC50predict_model.pkl"
joblib.dump(model, model_name)

# 결과 저장
results = [{
    'Normalized RMSE (A)': A,
    'Correct Ratio (B)': B,
    'Final Score': Score,
    'R^2': r2,
    'Adjusted R^2': adjusted_r2,
    'Pearson Correlation': pearson_corr,
    'max_depth': manual_params['max_depth'],
    'n_estimators': manual_params['n_estimators'],
    'learning_rate': manual_params['learning_rate'],
    'subsample': manual_params['subsample'],
    'colsample_bytree': manual_params['colsample_bytree'],
    'random_state': 42,
}]

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력
results_df


,Normalized RMSE (A),Correct Ratio (B),Final Score,R^2,Adjusted R^2,Pearson Correlation,max_depth,n_estimators,learning_rate,subsample,colsample_bytree,random_state
0,0.069089,0.759591,0.845251,0.822466,0.80551,0.907227,3,185,0.025159,0.608408,0.982354,42


In [60]:
import pandas as pd
import numpy as np
import joblib

# 모델 로드 (간결한 모델 이름 사용)
model_name = "XGBOOST_IC50predict_model.pkl"
model = joblib.load(model_name)

# 새로운 데이터 불러오기 (SMILES를 포함한 데이터)
new_df = df2

# 숫자형 데이터만 사용
numeric_df = new_df.select_dtypes(include=[np.number])

# 기존 모델과 동일한 전처리 적용
X_new = numeric_df.drop(columns=['IC50_nM', 'pIC50'], errors='ignore')

# 예측 수행 (pIC50으로 예측 후 IC50_nM으로 변환)
y_pred_pIC50 = model.predict(X_new)
y_pred_IC50_nM = 10 ** (-y_pred_pIC50+9)

# 예측 결과를 원본 데이터에 추가
new_df['IC50_nM'] = y_pred_IC50_nM

# 제출 파일 이름 간소화
submit_filename = 'IC50_Prediction_Submission'

# 결과를 CSV 파일로 저장
new_df[['ID', 'IC50_nM']].to_csv(f'{submit_filename}.csv', index=False)

print(f"{submit_filename}.csv 파일이 성공적으로 생성되었습니다.")


IC50_Prediction_Submission.csv 파일이 성공적으로 생성되었습니다.
